<a href="https://colab.research.google.com/github/ssubbinn/BAEKJOON/blob/main/%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 분류

지도학습의 대표적인 유형

지도학습 -> 명시적인 정답이 있는 데이터가 주어진 상태에서 학습하는 방식

- 학습 데이터로 주어진 데이터의 피처와 레이블 값을 머신러닝 알고리즘으로 학습해 모델 생성, 미지의 레이블값 예측

[앙상블]

1. 배깅

-> 랜덤 포레스트

2. 부스팅 (최근 발전)

-> XGBoost, LightGBM

[결정 트리]

: 데이터에 있는 규칙을 학습을 통해 자동으로 찾아내 트리 기반의 분류 규칙을 만드는 것

결정 트리 모델의 특징

결정트리의 장점

- 균일도라는 룰을 기반으로 하고 있어서 알고리즘이 쉽고 직관적임, 룰이 매우 명확

- 사전 데이터 가공 필요 없음

결정트리의 단점

- 과적합으로 정확도가 떨어짐, 트리의 크기를 사전에 제한해야 하는 튜닝 필요

-> DecisionTreeClassifier, DecisionTreeRegressor

파라미터

- min_samples_split : 노드를 분할하기 위한 최소한의 샘플 데이터 수, 과적합 제어에 사용, 디폴트 2이고 작게 설정할 수록 과적합 가능성 증가

- min_samples_leaf : 말단 노드가 되기 위한 최소한의 샘플 데이터 수, 과적합 제어 용도, 비대칭적 데이터에서 작게 설정 필요

- max_features : 최적의 분할을 위해 고려할 최대 피쳐 개수, 디폴트 = none 데이터의 모든 피처를 사용해 분할 수행

- max_depth : 트리의 최대 깊이를 규정, 디폴트는 none, none으로 설정하면 min_samples_split보다 작아질 때까지 계속 깊이 증가

- max_leaf_nodes : 말단 노드의 최대 개수

실습** 

### 앙상블 학습

[앙상블 학습 개요]

앙상블 학습의 유형: 보팅(Voting), 배깅(Bagging), 부스팅(Boosting)
- 보팅과 배깅은 여러 개의 분류기가 투표를 통해 최종 예측 결과를 결정하는 방식

보팅 -> 일반적으로 서로 다른 알고리즘을 가진 분류기가 결합하는 것

배깅 -> 각각의 분류기가 모두 같은 유형의 알고리즘 기반이지만 데이터 샘플링을 서로 다르게 가져가면서 학습을 수행해 보팅을 수행하는 것
- 대표적인 배깅 방식이 랜덤 포레스트 알고리즘

부스팅 -> 여러 개의 분류기가 순차적으로 학습을 수행하되, 앞에서 학습한 분류기가 예측이 틀린 데이터에 대해서는 올바르게 예측할 수 있도록 다음 분류기에는 가중치(weight)를 부여하면서 학습과 예측 진행

- XGBoost, LightGBM

[하드 보팅과 소프트 보팅]

1. 하드 보팅 -> 다수결 원칙
2. 소프트 보팅 -> 레이블 값 결정 확률을 모두 더하고 이를 평균해서 이들 중 가장 높은 레이블 값을 최종 보팅 결괏값으로 선정

In [8]:
# 보팅 분류기

import pandas as pd

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

cancer = load_breast_cancer()

data_df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
data_df.head(3)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758


In [9]:
#개별 모델은 로지스틱 회귀와 KNN임
lr_clf = LogisticRegression()
knn_clf = KNeighborsClassifier(n_neighbors=8)

#개별 모델을 소프트 보팅 기반의 앙상블 모델로 구현한 분류기
vo_clf = VotingClassifier(estimators=[('LR',lr_clf), ('KNN',knn_clf)], voting='soft')

X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, test_size=0.2, random_state=156)

#VotingClassifier 학습/예측/평가
vo_clf.fit(X_train, y_train)
pred = vo_clf.predict(X_test)
print('Voting 분류기 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

#개별 모델의 학습/예측/평가
classifiers = [lr_clf, knn_clf]
for classifier in classifiers:
    classifier.fit(X_train,y_train)
    pred = classifier.predict(X_test)
    class_name = classifier.__class__.__name__
    print('{0} 정확도: {1:.4f}'.format(class_name, accuracy_score(y_test,pred)))

Voting 분류기 정확도: 0.9474
LogisticRegression 정확도: 0.9386
KNeighborsClassifier 정확도: 0.9386


- 보팅 분류기 성능이 좋음. 항상 그렇지는 않지만 보통 보팅, 배깅, 부스팅 하면 성능 좋아짐

### 랜덤 포레스트 - 배깅

- 여러 개의 결정 트리 분류기가 전체 데이터에서 배깅 방식으로 각자의 데이터를 샘플링해 개별적으로 학습을 수행한 뒤 최종적으로 모든 분류기가 보팅을 통해 예측 결정을 함
- 부트스트래핑 (Bootstrapping): 여러 개의 데이터 세트를 중첩되게 분리하는 것

[하이퍼 파라미터 튜닝]

- n_estimators: 랜덤 포레스트에서 결정 트리의 개수를 지정하며 default는 10개임. 많이 설정할수록 좋은 성능을 기대할 수 있지만 계속 증가시킨다고 성능이 무조건 향상되는 것은 아님. 또한 늘릴수록 학습 수행 시간이 오래 걸리는 것도 감안해야 함
- max_features는 결정 트리에 사용된 max_features 파라미터와 같음. 하지만 RandomForestClassifier의 기본 max_features는 'None'이 아니라 'auto', 즉 'sqrt'와 같음. 따라서 랜덤 포레스트의 트리를 분할하는 피처를 참조할 때 전체 피처가 아니라 sqrt(전체 피처 개수)만큼 참조함
- max_depth나 min_samples_leaf와 같이 결정 트리에서 과적합을 개선하기 위해 사용되는 파라미터가 랜덤 포레스트에도 똑같이 적용될 수 있음

### GBM

: 가중치 업데이트를 경사 하강법(Gradient Descent)이용해서 함

<경사하강법>

오류 값 = 실제 값- 예측 값 , 오류식 = y - F(x)를 최소화 하는 방향성을 가지고 반복적으로 가중치 값을 업데이트 하는 것

- 분류, 회귀 모두 가능

[하이퍼 파라미터 튜닝]

- loss: 경사 하강법에서 사용할 비용 함수를 지정. 특별한 이유가 없으면 기본값인 'deviance'를 그대로 적용

- learning_rate: GBM이 학습을 진행할 때마다 적용하는 학습률임. Weak learner가 순차적으로 오류 값을 보정해 나가는 데 적용하는 계수로 0~1 사이의 값을 지정할 수 있으며 기본값은 0.1. 너무 작은 값을 적용하면 업데이트되는 값이 작아져서 최소 오류 값을 찾아 예측 성능이 높아질 가능성이 높음. 많은 weak learner은 순차적인 반복이 필요해서 수행 시간이 오래 걸리고 또 너무 작게 설정하면 모든 weak learner의 반복이 완료되어도 최소 오류 값을 찾지 못할 수 있음. 반대로 큰 값을 적용하면 최소 오류 값을 찾지 못하고 그냥 지나쳐 버려 예측 성능이 떨어질 가능성이 높아지지만 빠른 수행이 가능함. 이러한 특성 때문에 learning_rate은 n_estimators와 상호 보완적으로 조합해 사용함. [learning_rate을 작게 하고 - n_estimators를 크게 하면 더 이상 성능이 좋아지지 않는 한계점까지는 예측 성능이 조금씩 좋아질 수 있음. 하지만 수행 시간이 너무 오래 걸리는 단점이 있으며 예측 성능 역시 현격히 좋아지지는 않음.]

- n_estimators: weak learner의 개수임. weak learner가 순차적으로 오류를 보정하므로 개수가 많을수록 예측 성능이 일정 수준까지는 좋아질 수 있음. 하지만 개수가 많을수록 수행 시간이 오래 걸림. 기본값은 100임

- subsample: weak learner가 학습에 사용하는 데이터의 샘플링 비율. 기본값은 1이며 이는 전체 학습 데이터를 기반으로 학습한다는 의미임. (0.5면 학습 데이터의 50%) 과적합이 염려되는 경우 subsample을 1보다 작은 값으로 설정

### XGBoost

- 성능 좋고 수행 시간 빠름
- 자체 과적합 규제
- 내장된 교차 검증
- 결손값 자체 처리
- Tree pruning : 분할 수를 최소화

[하이퍼 파라미터]

1. 부스터 파라미터

- `eta` [default=0.3, alias: learning_rate]: GBM의 학습률(learning rate)과 같은 파라미터. 0에서 1 사이의 값을 지정하며 부스팅 스텝을 반복적으로 수행할 때 업데이트되는 학습률 값. 파이썬 래퍼 기반의 xgboost를 이용할 경우 default는 0.3이고 사이킷런 래퍼 클래스를 이용할 경우 eta는 learning_rate 파라미터로 대체되며, default는 0.1임. 보통은 0.01~0.2 사이의 값을 선호
- `num_boost_rounds`: GBM의 n_estimators와 같은 파라미터
- `min_child_weight` [default=1]: 트리에서 추가적으로 가지를 나눌지를 결정하기 위해 필요한 데이터들의 weight 총합 min_child_weight이 클수록 분할을 자제.과적합을 조절하기 위해 사용됨
- `gamma `[default=0, alias: min_split_loss]: 트리의 리프 노드를 추가적으로 나눌지를 결정할 최소 손실 감소 값으로 해당 값보다 큰 손실(loss)이 감소된 경우에 리프 노드를 분할함. 값이 클수록 과적합 감소 효과가 있음
- `max_depth` [default=6]: 트리 기반 알고리즘의 max_depth와 같으며 0을 지정하면 깊이에 제한이 없음. max_depth가 높으면 특정 피처 조건에 특화되어 룰 조건이 만들어지므로 과적합 가능성이 높아지며 보통은 3~10 사이의 값을 적용함
- `sub_sample` [default=1]: GBM의 subsample과 동일하며 트리가 커져서 과적합되는 것을 제어하기 위해 데이터를 샘플링하는 비율을 지정함. sub_sample=0.5로 지정하면 전체 데이터의 절반을 트리를 생성하는 데 사용함. 0에서 1 사이의 값이 가능하나 일반적으로 0.5~1 사이의 값을 사용함
- `colsample_bytree` [default=1]: GBM의 max_features와 유사하며 tree 생성에 필요한 feature(column)를 임의로 샘플링 하는 데 사용됨. 매우 많은 feature가 있는 경우 과적합을 조정하는 데 적용함
- `lambda` [default=1, alias:reg_lambda] : L2 Regularization 적용 값으로 feature 개수가 많을 경우 적용을 검토하며 값이 클수록 과적합 감소 효과가 있음
- `alpha` : L1 Regularization 적용값으로 feature 개수가 많을 경우 적용을 검토하며 값이 클수록 감소 효과가 있음
- `scale_pos_weight` [default=1] : 특정 값으로 치우친 비대칭한 클래스로 구성된 데이터 세트의 균형을 유지하기 위한 파라미터

*과적합 문제가 심각하다면 다음과 같이 적용함

- eta 값을 낮춤(0.01~0.1) eta 값을 낮출 경우 num_round (또는 n_estimators)는 반대로 높여줘야 함
- max_depth 값을 낮춤
- min_child_weight 값을 높임
- gamma 값을 높임
- subsample과 colsample_by_tree를 조정하는 것도 트리가 너무 복잡하게 생성되는 것을 막아 과적합 문제에 도움이 될 수 있음


### LightGBM

- XGBoost 보다 빠름, 성능차이는 안남
- 적은 데이터셋에 적용할 경우 과적합 발생하기 쉬움 (10000건 이하)

- 리프 중심 트리 분할

<주요 파라미터>

- num_iterations [default=100] : 반복 수행하려는 트리의 개수를 지정함. 크게 지정할수록 예측 성능이 높아질수 있으나, 너무 크게 지정하면 오히려 과적합으로 성능이 저하 될 수 있음. Scikit-Learn GBM과 XGBoost의 Scikit-Learn 호환 클래스의 n_estimators와 같은 파라미터이므로 LightGBM의 Scikit-Learn 호환 클래스에서는 n_estimators로 이름이 변경됨
- learning_rate [default=0.1] : 0에서 1사이의 값을 지정하며 Boosting 스텝을 반복적으로 수행할 때 업데이트되는 학습롤값임. 일반적으로 n_estimators를 크게하고 learning_rate를 작게해서 예측 성능을 향상시킬 수 있으나, 마찬가지로 과적합(overfitting) 이슈와 학습 시간이 길어지는 부정적인 영향도 고려해야함. GBM, XGBoost의 learning_rate와 같은 파라미터임
- max_depth [default=-1] : 트리 기반 알고리즘의 max_depth와 같으며 0보다 작은 값을 지정하면 깊이에 제한이 없음. 지금까지 소개한 Depth Wise 방식의 트리와 다르게 LightGBM은 Leaf wise 기반이므로 깊이가 상대적으로 더 깊음
- min_data_in_leaf [default=20] : Decision Tree의 min_samples_leaf와 같은 파라미터임. 하지만 Scikit-Learn 래퍼 LightGBM 클래스인 LightGBMClassifier에서는 min_child_samples 파라미터로 이름이 변경됨. 최종 결정 클래스인 Leaf 노드가 되기 위해서 최소한으로 필요한 레코드(데이터) 수이며, 과적합을 제어하기 위한 파라미터임
- num_leaves [default=31] : 하나의 트리가 가질 수 있는 최대 Leaf 개수임
- boosting [default=gbdt] : Boosting 트리를 생성하는 알고리즘을 기술함
- gbdt : 일반적인 그레디언트 부스팅 결정트리
- rf : 랜덤포레스트
- bagging_fraction [default=1.0] : 트리가 커져서 과적합되는 것을 제어하기 위해서 데이터 샘플링하는 비율을 지정함. Scikit-Learn의 GBM과 XGBoost의 sub_sample 파라미터와 동일하기에 Scikit-Learn 래퍼 LightGBM인 LightGBMClassifier에서는 sub_sample로 동일하게 파라미터 이름이 변경됨
- feature_fraction [default=1.0] : 개별 트리를 학습할 때마다 무작위로 선택하는 feature의 비율이며 과적합을 막기 위해 사용됨. GBM의 max_features와 유사하며, XGBClassifier의 colsample_bytree와 똑같으므로 LightGBMClassifier에서는 동일하게 colsample_bytree로 변경됨
- lambda_l2 [default=0.0] : L2 Regulation 제어를 위한 값으로 feature 개수가 많을 경우 적용을 검토하며 값이 클수록 과적합 감소 효과가 있음. XGBClassifier의 reg_lambda와 동일하므로 LightGBMClassifier에서는 reg_lambda로 변경됨
- lambda_l1 [default=0.0] : L1 Regulation 제어를 위한 값으로 L2와 마찬가지로 과적합 제어를 위한 것이며, XGBClassifier의 reg_alpha와 동일하므로 LightGBMClassifier에서는 reg_alpha로 변경됨

<Learning Task 파라미터>

- objective: 최솟값을 가져야 할 손실함수를 정의하며 Xgboost의 objective 파라미터와 동일함. 애플리케이션 유형, 즉 회귀, 다중 클래스 분류, 이진 분류인지에 따라서 objective인 손실 함수가 지정됨

[하이퍼 파라미터 튜닝 방안]

- num_leaves: 개별 트리가 가질 수 있는 최대 리프의 개수이고 LightGBM 모델의 복잡도를 제어하는 주요 파라미터임. 일반적으로 num_leaves의 개수를 높이면 정확도가 높아지지만, 반대로 트리의 깊이가 깊어지고 모델이 복잡도가 커져 과적합 영향도가 커짐
- min_data_in_leaf: 사이킷런 래퍼 클래스에서는 min_child_samples로 이름이 바뀌며 과적합을 개선하기 위한 중요한 파라미터임. num_leaves와 학습 데이터의 크기에 따라 달라지지만, 보통 큰 값으로 설정하면 트리가 깊어지는 것을 방지함
- max_depth: 명시적으로 깊이의 크기를 제한하며 num_leaves, min_data_in_leaf와 결합해 과적합을 개선하는 데 사용함



- learning_rate을 작게 하면서 n_estimators를 크기 하는 것은 부스팅 계열 튜닝에서 가장 기본적인 튜닝 방안이므로 이를 적용하는 것도 좋음 (n_estimators를 너무 크게 하는 것은 과적합이므로 오히려 성능이 저하될 수 있음을 유념해야 함)
- 과적합을 제어하기 위해서 reg_lambda, reg_alpha와 같은 regularization을 적용하거나 학습 데이터에 사용할 피처의 개수나 데이터 샘플링 레코드 개수를 줄이기 위해 colsample_bytree, subsample 파라미터를 적용할 수 있음